In [ ]:
import pandas as pd
from web3 import Web3


from mainnet_launch.constants import ALL_AUTOPOOLS, ALL_CHAINS
from mainnet_launch.database.schema.full import AutopoolDeposit, AutopoolTransfer, AutopoolWithdrawal, Transactions
from mainnet_launch.database.schema.postgres_operations import get_full_table_as_df


Fetched 7,502 logs for <class 'web3._utils.datatypes.Transfer'> from 20,722,908 to 23,283,779 (2,560,872 blocks)
Fetched 7502 new Autopool transfers for autoETH on eth
Fetched 2,892 logs for <class 'web3._utils.datatypes.Transfer'> from 20,722,909 to 23,283,779 (2,560,871 blocks)
Fetched 2892 new Autopool transfers for balETH on eth
Fetched 2,501 logs for <class 'web3._utils.datatypes.Transfer'> from 20,722,910 to 23,283,779 (2,560,870 blocks)
Fetched 2501 new Autopool transfers for autoLRT on eth
Fetched 4,094 logs for <class 'web3._utils.datatypes.Transfer'> from 21,241,103 to 35,063,792 (13,822,690 blocks)
Fetched 4094 new Autopool transfers for baseETH on base
Fetched 1,103 logs for <class 'web3._utils.datatypes.Transfer'> from 21,718,586 to 23,283,779 (1,565,194 blocks)
Fetched 1103 new Autopool transfers for dineroETH on eth
Fetched 3,801 logs for <class 'web3._utils.datatypes.Transfer'> from 22,032,640 to 23,283,779 (1,251,140 blocks)
Fetched 3801 new Autopool transfers for auto

HTTPError: 503 Server Error: Service Unavailable for url: https://sonic-mainnet.g.alchemy.com/v2/7dyyKalTzbS0cySjdpnkr

In [3]:
a = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "eth_getLogs",
    "params": [
        {
            "address": "0xCb119265AA1195ea363D7A243aD56c73EA42Eb59",
            "fromBlock": "0x26bf08a",
            "toBlock": "0x2b83bca",
            "topics": ["0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"],
        }
    ],
}

In [7]:
int(a["params"][0]["fromBlock"], 16), int(a["params"][0]["toBlock"], 16)

(40628362, 45628362)